<a href="https://colab.research.google.com/github/nurchamidah/Self-Project/blob/main/Precipitation_Data_Processing/Precip_Get_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For get an API key:
- Get a temporary email from https://temp-mail.org/id
- After you get the temporary email, register in https://www.worldweatheronline.com/weather-api/my/
- Go to my account and you can get the API key

In [ ]:
import os
import requests
import json
import pandas as pd
import glob

# bagian yang perlu diubah
frequency = 24
tahunMulai = 2013
tahunTerakhir = 2023
location_list = "3,117.16667"
api_key = '0e0626abcdd6407792112537240808'

#catatan
'''bulungan'''
latitude = 3
longitude = 117.16667

# tahun
for tahun in range(tahunMulai, tahunTerakhir + 1):
    # bulan dari januari (1) sampai desember (12)+1
    for bulan in range(1, 13):
        start_date = f"{tahun}-{bulan:02d}-01"  # tanggal mulai
        # 7 bulan pertama (januari-juli)
        if bulan <= 7:
            # bulan genap
            if bulan % 2 == 0:
                # bulan februari dan kabisat
                if bulan == 2 and tahun % 4 == 0:
                    end_date = f"{tahun}-{bulan:02d}-29"
                # bulan februari tidak kabisat
                elif bulan == 2 and tahun % 4 != 0:
                    end_date = f"{tahun}-{bulan:02d}-28"
                # bulan genap selain februari
                else:
                    end_date = f"{tahun}-{bulan:02d}-30"
            # selain februari
            else:
                end_date = f"{tahun}-{bulan:02d}-31"
        # 5 bulan terakhir (agustus-desember)
        else:
            # bulan genap
            if bulan % 2 == 0:
                end_date = f"{tahun}-{bulan:02d}-31"
            # bulan ganjil
            else:
                end_date = f"{tahun}-{bulan:02d}-30"

        # link
        url_page = f"http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={api_key}&q={location_list}&date={start_date}&enddate={end_date}&tp={frequency}&format=json"

        # request (permintaan)
        response = requests.get(url_page)
        if response.status_code == 200:
            data = response.json()
            weather_data = data['data']['weather']
            data1 = pd.DataFrame(weather_data)

            # ambil isi hourly
            data2 = []
            for k in range(len(data1)):
                data2.append(data1.loc[k, 'hourly'][0])
            data2 = pd.DataFrame(data2)

            # penggabungan kolom tanggal dengan hourly
            data3 = data1[['date']].copy()
            data3 = data3.join(data2)

            # buat data csv
            data3.to_csv(f"({start_date})_({end_date}).csv", index=False)
        else:
            print(f"Failed to retrieve data for {start_date} to {end_date}")

# petunjuk bila proses perulangan selesai
print('perulangan selesai')

# penggabungan semua csv
df = pd.concat(map(pd.read_csv, glob.glob('*.csv')))

# urutkan sesuai tanggal
df = df.sort_values(by=['date'])

# rubah ke excel
df.to_excel('data.xlsx', index=False)


perulangan selesai
